In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Define assets and time range
gold = "GC=F"
stocks = "^GSPC"
crypto = "^CMC200"
# Gold, S&P 500 (Stocks), Bitcoin (Crypto)
start_date = "2019-01-01"
end_date = "2024-01-01"

# Fetch historical adjusted closing prices
gold_data = yf.download(gold, start=start_date, end=end_date)['Close']
stocks_data = yf.download(stocks, start=start_date, end=end_date)['Close']
crypto_data = yf.download(crypto, start=start_date, end=end_date)['Close']

# Missing values
gold_data = gold_data.dropna()
stocks_data = stocks_data.dropna()
crypto_data = crypto_data.dropna()

# Display first rows
print(gold_data.head())
print(stocks_data.head())
print(crypto_data.head())

# gold_data.to_csv('gold_data.csv')
# stocks_data.to_csv('stocks_data.csv')
# crypto_data.to_csv('crypto_data.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Ticker             GC=F
Date                   
2019-01-02  1281.000000
2019-01-03  1291.800049
2019-01-04  1282.699951
2019-01-07  1286.800049
2019-01-08  1283.199951
Ticker            ^GSPC
Date                   
2019-01-02  2510.030029
2019-01-03  2447.889893
2019-01-04  2531.939941
2019-01-07  2549.689941
2019-01-08  2574.409912
Ticker         ^CMC200
Date                  
2019-01-02  107.209999
2019-01-03  103.699997
2019-01-04  104.480003
2019-01-07  107.930000
2019-01-08  108.279999


In [ ]:
print(type(gold_data), len(gold_data))
print(type(stocks_data), len(stocks_data))
print(type(crypto_data), len(crypto_data))

<class 'pandas.core.frame.DataFrame'> 1258
<class 'pandas.core.frame.DataFrame'> 1258
<class 'pandas.core.frame.DataFrame'> 1231


In [ ]:
print(gold_data.index)
print(stocks_data.index)
print(crypto_data.index)

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=1258, freq=None)
DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='Date', length=1258, freq=None)
DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',

In [ ]:
# Plotting data

# gold_data.plot()
# stocks_data.plot()
# crypto_data.plot()

In [ ]:
# Day by day returns
gold_returns = gold_data.pct_change()
stock_returns = stocks_data.pct_change()
crypto_returns = crypto_data.pct_change()


# gold_returns.plot()
# stock_returns.plot()
# crypto_returns.plot()

In [ ]:
#joins all three datasets, drops all the dates with missing values
data= pd.merge(pd.merge(gold_data,stocks_data,on='Date'),crypto_data,on='Date').dropna()
daily_returns = data.pct_change().dropna()

In [ ]:
#risk score calculation
answer_score =45
# this score will be from the questions in the pdf in discord
def calc_risk_score_Grable_Lytton(y):
    if 0 <= y <= 18:
        return (0.2 / 18) * y
    elif 18 < y <= 22:
        return 0.2 + ((0.4 - 0.2) / (22 - 18)) * (y - 18)
    elif 22 < y <= 28:
        return 0.4 + ((0.6 - 0.4) / (28 - 22)) * (y - 22)
    elif 28 < y <= 32:
        return 0.6 + ((0.8 - 0.6) / (32 - 28)) * (y - 28)
    elif 32 < y <= 45:
        return 0.8 + ((1.0 - 0.8) / (45 - 32)) * (y - 32)
    else:
        return None

risk_score= round(calc_risk_score_Grable_Lytton(answer_score),2)

In [ ]:
annual_returns = daily_returns.mean() * 252  # Assuming 252 trading days in a year

#  Calculate the Covariance Matrix:
covariance_matrix = daily_returns.cov() * 252 # Annualized covariance
from scipy.optimize import minimize
import numpy as np

def optimize_portfolio(risk_score, annual_returns, covariance_matrix):
    """
    Optimizes portfolio allocation based on user risk score using Mean-Variance Optimization.

    risk_score: float (0 to 1) - User's risk preference
    annual_returns: Series - Annualized expected returns of assets
    covariance_matrix: DataFrame - Covariance matrix of asset returns

    Returns: Optimal portfolio weights for Gold, Stocks, Crypto
    """
    # Convert to NumPy arrays
    mu = annual_returns.values
    cov_matrix = covariance_matrix.values

    # Dynamic risk aversion coefficient (higher value = more conservative)
    lambda_val = 10 - 9 * risk_score

    # Objective function (negative because we maximize returns)
    def objective(w):
        return - (np.dot(w, mu) - lambda_val * np.dot(w.T, np.dot(cov_matrix, w)))

    # Constraints: Weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

    # Bounds: No short selling (weights between 0 and 1)
    bounds = [(0, 1)] * len(mu)

    # Initial guess (equal allocation)
    w0 = np.array([1/3, 1/3, 1/3])

    # Solve optimization problem
    result = minimize(objective, w0, bounds=bounds, constraints=constraints)

    # Extract optimal weights
    optimal_weights = result.x
    return optimal_weights

# Example usage with a user having risk_score of
# risk_score = 1
optimal_weights = optimize_portfolio(risk_score, annual_returns, covariance_matrix)

# Print results
assets = ["Gold", "Stocks", "Crypto"]
allocation = dict(zip(assets, optimal_weights))
print("Optimal Portfolio Allocation:", allocation)


Optimal Portfolio Allocation: {'Gold': 0.18958429018390283, 'Stocks': 0.27253239570010934, 'Crypto': 0.5378833141159879}


Sentiment Analysis

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from transformers import pipeline

In [ ]:
tickers = ["GC=F", "BTC-USD", "ETH-USD", "XRP-USD", "^GSPC"]  # Gold Futures, Bitcoin, S&P 500

all_links = {}  # Dictionary to store links for each ticker

for ticker in tickers:
    stock = yf.Ticker(ticker)
    news_list = stock.news  # Fetch news data
    ticker_links = [] # List to store links for the current ticker

    if news_list:
        for article in news_list[:3]:  # Get latest 3 articles
            content = article.get("content", {})  # Extract 'content' field
            title = content.get("title", "No Title")
            # print(content)
            # Handle NoneType safely
            link = (content.get("clickThroughUrl") or {}).get("url", "No Link")
            ticker_links.append(link) # Add the link to the current ticker's list
            # print(f"{title} - {link}")
    else:
        print(f"No news found for {ticker}.")

    all_links[ticker] = ticker_links # Add the ticker's links to the main dictionary

# Print the dictionary containing all links
print("\nAll Links:")
for ticker, links in all_links.items():
    print(f"{ticker}: {links}")

# Example of accessing all links across all tickers
print("\nAll Links Across All Tickers:")
for ticker, links in all_links.items():
    for link in links:
      if link != "No Link":
        print(link)


All Links:
GC=F: ['https://finance.yahoo.com/news/pound-gold-oil-prices-commodity-currency-check-093020952.html', 'https://finance.yahoo.com/news/gold-prices-prediction-trade-tariffs-poll-result-161541235.html', 'https://finance.yahoo.com/news/pound-gold-oil-prices-commodity-currency-095723302.html']
BTC-USD: ['No Link', 'No Link', 'https://finance.yahoo.com/news/crypto-booming-most-financial-advisors-130002418.html']
ETH-USD: ['https://finance.yahoo.com/video/coinbase-posts-q4-revenue-beat-212030484.html', 'https://finance.yahoo.com/news/ether-close-three-month-lows-131239368.html', 'https://finance.yahoo.com/video/effect-peer-pressure-among-financial-120022156.html']
XRP-USD: ['No Link', 'https://finance.yahoo.com/video/coinbase-revenue-spikes-crypto-undergoes-153405622.html', 'https://finance.yahoo.com/video/coinbase-posts-q4-revenue-beat-212030484.html']
^GSPC: ['https://finance.yahoo.com/video/larger-companies-dominate-ipo-market-190033685.html', 'https://finance.yahoo.com/news/s

In [ ]:
# Initialize an empty list to store the valid links
valid_links = []

# Loop through all tickers and their links
for links in all_links.values():
    for link in links:
        if link != "No Link":  # Filter out "No Link"
            valid_links.append(link)

# Print the list of valid links
print(valid_links)

['https://finance.yahoo.com/news/pound-gold-oil-prices-commodity-currency-check-093020952.html', 'https://finance.yahoo.com/news/gold-prices-prediction-trade-tariffs-poll-result-161541235.html', 'https://finance.yahoo.com/news/pound-gold-oil-prices-commodity-currency-095723302.html', 'https://finance.yahoo.com/news/crypto-booming-most-financial-advisors-130002418.html', 'https://finance.yahoo.com/video/coinbase-posts-q4-revenue-beat-212030484.html', 'https://finance.yahoo.com/news/ether-close-three-month-lows-131239368.html', 'https://finance.yahoo.com/video/effect-peer-pressure-among-financial-120022156.html', 'https://finance.yahoo.com/video/coinbase-revenue-spikes-crypto-undergoes-153405622.html', 'https://finance.yahoo.com/video/coinbase-posts-q4-revenue-beat-212030484.html', 'https://finance.yahoo.com/video/larger-companies-dominate-ipo-market-190033685.html', 'https://finance.yahoo.com/news/stocks-near-record-highs-as-investors-await-fed-minutes-manufacturing-update-what-to-know-

In [ ]:
sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def scrape_article(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}  # Mimic a real browser
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # Yahoo Finance articles usually have content inside <p> tags
        paragraphs = soup.find_all("p")
        article_text = " ".join([p.get_text() for p in paragraphs])

        return article_text if article_text else "No article content found."

    except Exception as e:
        return f"Error scraping article: {str(e)}"

In [ ]:
full_text = ""
for valid_link in valid_links:
  full_text = full_text + scrape_article(valid_link)
print(full_text)

The pound moved higher against the dollar in early European trading, trading just below $1.26, after registering its best week of the year. The currency’s upward movement was spurred by a positive UK Gross Domestic Product (GDP) report and disappointing US retail sales data. The dollar's weakness was further compounded by a general market shift, as US markets remain closed on Monday for President's Day. Read more: FTSE 100 LIVE: London markets tick up as Starmer 'ready' to send peacekeeping troops to Ukraine Additionally, analysts noted that growing concerns over tariffs were having less of an impact on investor sentiment. "It seems that each successive tariff announcement seems to be having less of an impact on markets," analysts at Lloyds said. "Markets seemed more interested in Trump's announcement that he had talks on Ukraine with Russian leader Putin. "That has helped drive a bounce in both the euro and sterling against the dollar to their highest in around three weeks, despite th

In [ ]:
def analyze_sentiment(article_text):
    result = sentiment_pipeline(article_text[:512])  # Truncate to 512 tokens if needed
    return result


In [ ]:
import re
assets = {
    "gold": ["gold", "precious metal", "bullion"],
    "crypto": ["bitcoin", "crypto", "cryptocurrency", "ethereum", "XRP"],
    "stocks": ["stocks", "equities", "S&P 500", "Dow Jones"]
}

def analyze_asset_sentiment(article_text):
    asset_sentiment = {asset: [] for asset in assets}  # Store sentiments per asset

    sentences = re.split(r"(?<=[.!?]) +", article_text)  # Split into sentences

    for sentence in sentences:
        for asset, keywords in assets.items():
            if any(keyword.lower() in sentence.lower() for keyword in keywords):
                result = sentiment_pipeline(sentence[:512])  # Truncate if needed
                asset_sentiment[asset].append(result[0]["label"])

    return asset_sentiment
sentiment_result = analyze_asset_sentiment(full_text)
print(sentiment_result)

{'gold': ['positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'negative'], 'crypto': ['neutral', 'negative', 'positive', 'negative', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral'], 'stocks': ['positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral']}


In [ ]:
# Initialize a dictionary to store the scores
sentiment_scores = {}

# Define a function to map sentiments to scores
sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}

# Loop through each category and calculate the score
for category, sentiments in sentiment_result.items():
    score = sum(sentiment_mapping[sentiment] for sentiment in sentiments)
    sentiment_scores[category] = score

# Print the sentiment scores
print(sentiment_scores)


{'gold': 5, 'crypto': 3, 'stocks': 9}


Individual Models